In [1]:
import os
import sys
import gc
import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from multiprocessing import cpu_count, Pool

In [2]:
pd.options.display.max_columns = None

In [3]:
PATH = os.path.join('..', 'feature')

KEY = 'card_id'

files = os.listdir(PATH)

test = pd.read_csv(os.path.join('..', 'data', 'test.csv'))

features = ['f105.pkl', 'f109.pkl', 'f111.pkl']
features += ['f107_N.pkl', 'f107_Y.pkl',
             'f108_N.pkl', 'f108_Y.pkl',
             'f110_N.pkl', 'f110_Y.pkl']
features += ['f205.pkl', 'f209.pkl']
features += ['f207_N.pkl', 'f207_Y.pkl',
             'f208_N.pkl', 'f208_Y.pkl',
             'f210_N.pkl', 'f210_Y.pkl']
features += ['f302.pkl', 'f303.pkl', 'f304.pkl', 'f305.pkl', 'f306.pkl']
features += ['f403.pkl', 'f404.pkl', 'f409.pkl', 'f411.pkl']
features += ['f406_N.pkl', 'f406_Y.pkl',
             'f407_N.pkl', 'f407_Y.pkl',
             'f408_N.pkl', 'f408_Y.pkl']

for f in features:
    print(f'Merge: {f}', end=' ')
    test = pd.merge(test, pd.read_pickle(os.path.join('..', 'feature', f)), on=KEY, how='left')
    print('Done!!')

Merge: f105.pkl Done!!
Merge: f109.pkl Done!!
Merge: f111.pkl Done!!
Merge: f107_N.pkl Done!!
Merge: f107_Y.pkl Done!!
Merge: f108_N.pkl Done!!
Merge: f108_Y.pkl Done!!
Merge: f110_N.pkl Done!!
Merge: f110_Y.pkl Done!!
Merge: f205.pkl Done!!
Merge: f209.pkl Done!!
Merge: f207_N.pkl Done!!
Merge: f207_Y.pkl Done!!
Merge: f208_N.pkl Done!!
Merge: f208_Y.pkl Done!!
Merge: f210_N.pkl Done!!
Merge: f210_Y.pkl Done!!
Merge: f302.pkl Done!!
Merge: f303.pkl Done!!
Merge: f304.pkl Done!!
Merge: f305.pkl Done!!
Merge: f306.pkl Done!!
Merge: f403.pkl Done!!
Merge: f404.pkl Done!!
Merge: f409.pkl Done!!
Merge: f411.pkl Done!!
Merge: f406_N.pkl Done!!
Merge: f406_Y.pkl Done!!
Merge: f407_N.pkl Done!!
Merge: f407_Y.pkl Done!!
Merge: f408_N.pkl Done!!
Merge: f408_Y.pkl Done!!


In [4]:
columns = [c for c in test.columns if str(test[c].dtypes) != 'object']

In [5]:
len(columns)

1156

In [10]:
from collections import Counter

In [11]:
cnt = Counter(columns)
cnt.most_common()

[('N_authorized_flag_x', 3),
 ('Y_authorized_flag_x', 3),
 ('N_authorized_flag_y', 3),
 ('Y_authorized_flag_y', 3),
 ('union_transactions_count_x', 2),
 ('union_transactions_count_y', 2),
 ('feature_1', 1),
 ('feature_2', 1),
 ('feature_3', 1),
 ('mean_feature_1', 1),
 ('mean_feature_2', 1),
 ('mean_feature_3', 1),
 ('first_active_month_year', 1),
 ('first_active_month_weekday', 1),
 ('first_active_month_month', 1),
 ('first_active_month_dayofweek', 1),
 ('first_active_month_weekofyear', 1),
 ('elapsed_time', 1),
 ('union_year_nunique_x', 1),
 ('union_weekofyear_nunique_x', 1),
 ('union_month_nunique_x', 1),
 ('union_dayofweek_nunique_x', 1),
 ('union_weekend_nunique_x', 1),
 ('union_hour_nunique_x', 1),
 ('hist_transactions_count', 1),
 ('hist_category_1_sum', 1),
 ('hist_category_1_mean', 1),
 ('hist_category_1_nunique', 1),
 ('hist_category_2_nunique', 1),
 ('hist_category_3_nunique', 1),
 ('hist_merchant_id_nunique', 1),
 ('hist_purchase_amount_sum', 1),
 ('hist_purchase_amount_mea

In [8]:
for i in columns:
    print(i, test[i].dtypes)

feature_1 int64
feature_2 int64
feature_3 int64
mean_feature_1 float64
mean_feature_2 float64
mean_feature_3 float64
first_active_month_year int64
first_active_month_weekday int64
first_active_month_month int64
first_active_month_dayofweek int64
first_active_month_weekofyear int64
elapsed_time int64
union_transactions_count_x union_transactions_count_x    int16
union_transactions_count_x    int16
dtype: object
union_year_nunique_x int8
union_weekofyear_nunique_x int8
union_month_nunique_x int8
union_dayofweek_nunique_x int8
union_weekend_nunique_x int8
union_hour_nunique_x int8
hist_transactions_count int16
hist_category_1_sum int16
hist_category_1_mean float16
hist_category_1_nunique int8
hist_category_2_nunique int8
hist_category_3_nunique int8
hist_merchant_id_nunique int16
hist_purchase_amount_sum float32
hist_purchase_amount_mean float32
hist_purchase_amount_max float32
hist_purchase_amount_min float16
hist_purchase_amount_std float32
hist_installments_sum int16
hist_installments_

hist_N_installments_median_std_x float16
hist_N_installments_median_var_x float32
hist_N_installments_median_skew_x float16
hist_N_installments_std_mean_x float16
hist_N_installments_std_std_x float16
hist_N_installments_std_var_x float32
hist_N_installments_std_skew_x float16
hist_N_installments_var_mean_x float32
hist_N_installments_var_std_x float32
hist_N_installments_var_var_x float32
hist_N_installments_var_skew_x float16
hist_N_installments_skew_mean_x float16
hist_N_installments_skew_std_x float16
hist_N_installments_skew_var_x float16
hist_N_installments_skew_skew_x float16
hist_N_installments_count_mean_x float16
hist_N_installments_count_std_x float16
hist_N_installments_count_var_x float32
hist_N_installments_count_skew_x float16
hist_Y_month_lag_mean_x float16
hist_Y_month_lag_std_x float16
hist_Y_month_lag_var_x float16
hist_Y_month_lag_skew_x float16
hist_Y_purchase_amount_min_mean_x float16
hist_Y_purchase_amount_min_std_x float16
hist_Y_purchase_amount_min_var_x float1

hist_N_purchase_amount_min_skew_y float64
hist_N_purchase_amount_max_mean_y float64
hist_N_purchase_amount_max_std_y float64
hist_N_purchase_amount_max_var_y float64
hist_N_purchase_amount_max_skew_y float64
hist_N_purchase_amount_mean_mean_y float64
hist_N_purchase_amount_mean_std_y float64
hist_N_purchase_amount_mean_var_y float64
hist_N_purchase_amount_mean_skew_y float64
hist_N_purchase_amount_median_mean_y float64
hist_N_purchase_amount_median_std_y float64
hist_N_purchase_amount_median_var_y float64
hist_N_purchase_amount_median_skew_y float64
hist_N_purchase_amount_std_mean_y float64
hist_N_purchase_amount_std_std_y float64
hist_N_purchase_amount_std_var_y float64
hist_N_purchase_amount_std_skew_y float64
hist_N_purchase_amount_var_mean_y float64
hist_N_purchase_amount_var_std_y float64
hist_N_purchase_amount_var_var_y float64
hist_N_purchase_amount_var_skew_y float64
hist_N_purchase_amount_skew_mean_y float64
hist_N_purchase_amount_skew_std_y float64
hist_N_purchase_amount_skew

merchants_avg_purchases_lag6_var float32
merchants_avg_purchases_lag6_skew float16
merchants_active_months_lag6_nunique int8
merchants_active_months_lag6_min float16
merchants_active_months_lag6_max float16
merchants_active_months_lag6_mean float16
merchants_active_months_lag6_median float16
merchants_active_months_lag6_std float16
merchants_active_months_lag6_var float16
merchants_active_months_lag6_skew float16
merchants_avg_sales_lag12_nunique int16
merchants_avg_sales_lag12_min float16
merchants_avg_sales_lag12_max float32
merchants_avg_sales_lag12_mean float32
merchants_avg_sales_lag12_median float16
merchants_avg_sales_lag12_std float32
merchants_avg_sales_lag12_var float32
merchants_avg_sales_lag12_skew float16
merchants_avg_purchases_lag12_nunique int16
merchants_avg_purchases_lag12_min float16
merchants_avg_purchases_lag12_max float64
merchants_avg_purchases_lag12_mean float64
merchants_avg_purchases_lag12_median float16
merchants_avg_purchases_lag12_std float16
merchants_avg_

union_N_purchase_amount_mean_std float32
union_N_purchase_amount_mean_var float32
union_N_purchase_amount_mean_skew float16
union_N_purchase_amount_median_mean float32
union_N_purchase_amount_median_std float32
union_N_purchase_amount_median_var float32
union_N_purchase_amount_median_skew float16
union_N_purchase_amount_std_mean float16
union_N_purchase_amount_std_std float32
union_N_purchase_amount_std_var float32
union_N_purchase_amount_std_skew float16
union_N_purchase_amount_var_mean float32
union_N_purchase_amount_var_std float32
union_N_purchase_amount_var_var float32
union_N_purchase_amount_var_skew float16
union_N_purchase_amount_skew_mean float16
union_N_purchase_amount_skew_std float16
union_N_purchase_amount_skew_var float16
union_N_purchase_amount_skew_skew float16
union_N_purchase_amount_count_mean float16
union_N_purchase_amount_count_std float16
union_N_purchase_amount_count_var float32
union_N_purchase_amount_count_skew float16
union_N_installments_min_mean float16
unio

In [ ]:
drop = [
    'N_authorized_flag_x',
    'Y_authorized_flag_x',
    'N_authorized_flag_y',
    'Y_authorized_flag_y',
    'union_transactions_count_x',
    'union_transactions_count_y', 
]

for f in [
    'hist_purchase_date_max', 'hist_purchase_date_min',
    'N_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_min',
    'Y_hist_auth_purchase_date_max', 'Y_hist_auth_purchase_date_min',
    'new_purchase_date_max', 'new_purchase_date_min',
    'N_new_auth_purchase_date_max', 'N_new_auth_purchase_date_min',
    'Y_new_auth_purchase_date_max', 'Y_new_auth_purchase_date_min',
    'union_purchase_date_max', 'union_purchase_date_min',
    'N_union_auth_purchase_date_max', 'N_union_auth_purchase_date_min',
    'Y_union_auth_purchase_date_max', 'Y_union_auth_purchase_date_min']:
    test[f] = test[f].astype(np.int64) * 1e-9


# test = test.drop(drop_col, axis=1)

drop = []
columns = test.columns[1:]
for i in tqdm(range(0, len(columns))):
    for j in range(i+1, len(columns)):
        if sum(test[columns[i]] != test[columns[j]]) == 0:
            drop.append(columns[j])

print(drop)

In [6]:
test = test[columns]

In [12]:
for f in [
    'hist_purchase_date_max', 'hist_purchase_date_min', 
    'N_hist_auth_purchase_date_max', 'N_hist_auth_purchase_date_min', 
    'Y_hist_auth_purchase_date_max', 'Y_hist_auth_purchase_date_min', 
    'new_purchase_date_max', 'new_purchase_date_min', 
    'N_new_auth_purchase_date_max', 
    'N_new_auth_purchase_date_min',
    'Y_new_auth_purchase_date_max', 'Y_new_auth_purchase_date_min', 
    'union_purchase_date_max', 
    'union_purchase_date_min', 
    'N_union_auth_purchase_date_max',
    'N_union_auth_purchase_date_min',
    'Y_union_auth_purchase_date_max', 
    'Y_union_auth_purchase_date_min'
]:
    test[f] = test[f].astype(np.int64) * 1e-9

test = test.drop(['N_authorized_flag_x', 'Y_authorized_flag_x',
                  'N_authorized_flag_y', 'Y_authorized_flag_y',
                  'union_transactions_count_x', 'union_transactions_count_y'], axis=1)

In [13]:
# test.shape

In [14]:
drop = []
columns = test.columns
l = len(columns)
for i in tqdm(range(0, l)):
    for j in range(i+1, l):
        if sum(test[columns[i]] != test[columns[j]]) == 0:
            drop.append(columns[j])

100%|██████████| 904/904 [29:09<00:00,  1.93s/it] 


In [ ]:
drop

In [15]:
test = test[drop]

In [16]:
test.head(20)

,first_active_month_dayofweek,hist_installments_count_mean,hist_installments_count_std,hist_installments_count_var,Y_union_transactions_count_x,Y_hist_auth_transactions_count,historica_merchants_transactions_count,hist_purchase_date_diff,union_purchase_date_min,union_month_lag_min,Y_hist_auth_transactions_count,historica_merchants_transactions_count,N_union_auth_purchase_date_max,N_union_auth_purchase_date_min,historica_merchants_transactions_count,Y_hist_auth_purchase_date_diff,Y_union_auth_purchase_date_min,Y_union_auth_month_lag_min,Y_new_auth_purchase_date_max,Y_new_auth_purchase_date_min,N_new_auth_purchase_date_min,Y_union_transactions_count,Y_union_auth_transactions_count,merchants_active_months_lag3_median,union_installments_count_mean,union_installments_count_std,union_installments_count_var,union_purchase_date_diff,Y_union_auth_transactions_count,Y_union_auth_purchase_date_diff
0,5,7.554688,3.574219,12.777778,68,68,68,268,1.491330e+09,-8,68,68,1.513186e+09,1.493303e+09,68,268,1.491330e+09,-8,1.519845e+09,1.517651e+09,-9.223372e+09,71,71,3.0,7.101562,3.666016,13.433333,330,71,330
1,6,13.000000,7.015625,49.200001,78,78,78,401,1.484321e+09,-13,78,78,1.494079e+09,1.494079e+09,78,401,1.484321e+09,-13,1.524247e+09,1.520080e+09,-9.223372e+09,87,87,3.0,10.875000,7.121094,50.696430,462,87,462
2,1,2.166016,0.983398,0.966667,13,13,13,161,1.503673e+09,-6,13,13,1.517593e+09,1.515681e+09,13,161,1.503673e+09,-6,1.520947e+09,1.519916e+09,-9.223372e+09,15,15,3.0,2.142578,0.899902,0.809524,199,15,199
3,4,8.664062,6.507812,42.333332,26,26,26,77,1.512392e+09,-2,26,26,-9.223372e+09,-9.223372e+09,26,77,1.512392e+09,-2,1.524000e+09,1.520162e+09,-9.223372e+09,36,36,3.0,7.199219,5.214844,27.200001,134,36,134
4,1,7.855469,6.113281,37.362637,110,110,110,419,1.483444e+09,-13,110,110,1.514992e+09,1.483444e+09,110,418,1.483554e+09,-13,1.523535e+09,1.520132e+09,-9.223372e+09,116,116,3.0,7.250000,5.972656,35.666668,464,116,462
5,5,2.285156,1.379883,1.904762,16,16,16,231,1.499828e+09,-7,16,16,1.514931e+09,1.501690e+09,16,231,1.499828e+09,-7,1.520852e+09,1.520852e+09,-9.223372e+09,17,17,3.0,2.125000,1.356445,1.839286,243,17,243
6,6,9.601562,4.097656,16.799999,48,48,48,138,1.507572e+09,-4,48,48,-9.223372e+09,-9.223372e+09,48,138,1.507572e+09,-4,1.524562e+09,1.520019e+09,-9.223372e+09,57,57,3.0,8.140625,4.257812,18.142857,196,57,196
7,2,2.500000,2.507812,6.285714,20,20,20,350,1.486464e+09,-12,20,20,1.516717e+09,1.504967e+09,20,350,1.486464e+09,-12,-9.223372e+09,-9.223372e+09,-9.223372e+09,20,20,3.0,2.500000,2.507812,6.285714,350,20,350
8,5,6.875000,5.742188,32.982143,55,55,55,234,1.499527e+09,-7,55,55,1.519811e+09,1.503577e+09,55,203,1.499527e+09,-7,1.524899e+09,1.521300e+09,-9.223372e+09,57,57,3.0,5.699219,5.636719,31.788889,293,57,293
9,1,6.855469,3.759766,14.142858,48,48,48,265,1.483614e+09,-8,48,48,1.506568e+09,1.484733e+09,48,265,1.483614e+09,-8,1.511060e+09,1.507467e+09,-9.223372e+09,57,57,3.0,6.332031,3.427734,11.750000,317,57,317


In [17]:
h = test.head(20).reset_index(drop=True).fillna(0)

In [18]:
h.dtypes

first_active_month_dayofweek                int64
hist_installments_count_mean              float16
hist_installments_count_std               float16
hist_installments_count_var               float32
Y_union_transactions_count_x                int16
Y_hist_auth_transactions_count              int16
historica_merchants_transactions_count      int16
hist_purchase_date_diff                     int16
union_purchase_date_min                   float64
union_month_lag_min                          int8
Y_hist_auth_transactions_count              int16
historica_merchants_transactions_count      int16
N_union_auth_purchase_date_max            float64
N_union_auth_purchase_date_min            float64
historica_merchants_transactions_count      int16
Y_hist_auth_purchase_date_diff              int16
Y_union_auth_purchase_date_min            float64
Y_union_auth_month_lag_min                   int8
Y_new_auth_purchase_date_max              float64
Y_new_auth_purchase_date_min              float64


In [19]:
for ci in h.columns:
    for cj in h.columns:
        print(h[cj].dtypes)
        if ci == cj:
            continue
        if sum(h[ci] != h[cj]) == 0:
            print(ci, cj)

int64
float16
float16
float32
int16
Y_hist_auth_transactions_count    int16
Y_hist_auth_transactions_count    int16
dtype: object


TypeError: '<' not supported between instances of 'str' and 'int'